In [ ]:
!pip install anthropic

In [2]:
from anthropic import Anthropic
import os

In [3]:
from google.colab import userdata


In [4]:
client = Anthropic(
    api_key = userdata.get('ANTHROPIC_API_KEY')
)

In [5]:
test = {
    "role" : "user",
    "content": "What is the current temperature in Delhi"
}

message = client.messages.create(
    model = "claude-3-opus-20240229",
    max_tokens = 1024,
    temperature = 0.1,
    messages = [test]
)

print(message.content[0])

ContentBlock(text='I apologize, but I do not have access to real-time weather data. As an AI language model, I don\'t have the ability to retrieve current weather information for specific locations. However, you can easily find the current temperature in Delhi by using a weather app or website that provides real-time weather updates, such as Google Weather, AccuWeather, or The Weather Channel. Simply search for "Delhi weather" on any of these platforms to get the most up-to-date information on the current temperature and other weather conditions in the city.', type='text')


In [6]:
def dollar_price(currency):
  return 82.75

In [7]:
func = """
{
 "tool_description": {
      "tool_name" : "dollar_price",
      "description" : "Gives the latest dollar price",
      "parameters":{
          "currency": {
            "type" : string
          }
      }
 }
}
"""

In [8]:
system_prompt = """
You have access to an additional set of one or more functions you can use to answer the user's question.

You may call them like this:
{{
  "function_calls":{{
    "invoke": [
      {{"tool_name": $TOOL_NAME,
      "parameters": $PARAMETER_VALUE}},

      {{"tool_name": $TOOL_NAME,
      "parameters": $PARAMETER_VALUE}}
    ]
  }}
}}

Here are the tools available:
{{
  "tools":{func}
}}
Your final answer should be in the form the "The current dollar price is {{price}} {{currency}}

""".format(func=func)

print(system_prompt)



You have access to an additional set of one or more functions you can use to answer the user's question.

You may call them like this:
{
  "function_calls":{
    "invoke": [
      {"tool_name": $TOOL_NAME,
      "parameters": $PARAMETER_VALUE},

      {"tool_name": $TOOL_NAME,
      "parameters": $PARAMETER_VALUE}
    ]
  }
}

Here are the tools available:
{
  "tools":
{
 "tool_description": {
      "tool_name" : "dollar_price",
      "description" : "Gives the latest dollar price",
      "parameters":{
          "currency": {
            "type" : string
          }
      }
 }
}

}
Your final answer should be in the form the "The current dollar price is {price} {currency}




In [9]:
message = client.messages.create(
    model = "claude-3-opus-20240229",
    max_tokens = 1024,
    temperature = 0.0,
    system = system_prompt,
    messages = [
        {"role": "user",
        "content": "What is the current dollar price in INR"}]
)

print(message.content[0].text)

{
  "function_calls":{
    "invoke": [
      {"tool_name": "dollar_price",
      "parameters": {"currency": "INR"}}
    ]
  }
}


In [11]:
import json
def info(sample):
  format = json.loads(sample)
  currency = format['function_calls']['invoke'][0]['parameters']['currency']
  tool_name = format['function_calls']['invoke'][0]['tool_name']
  return tool_name,currency

In [12]:
tool_name,currency = info(message.content[0].text)

In [26]:
def construct_injection_prompt(invoke_results):
  construct_prompt = """
  {{
    "function_results": {{
      "results": [
        {tool_name}
      ]
    }}
  }}
  """


  res_strings = [json.dumps({

    "tool_name": res['tool_name'],
    "tool_result": res['tool_result']}) for res in invoke_results]

  res_list_string = "".join(res_strings).strip()

  construct_prompt = construct_prompt.format(tool_name=res_list_string)

  return construct_prompt

formatted_results = [{
    'tool_name': tool_name,
    'tool_result': currency
}]

func_results = construct_injection_prompt(formatted_results)


In [30]:
partial_assistant_message = message.content[0].text + "{function_calls}" + func_results

final_message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "What is the current price of dollar in INR"
        },
        {
            "role": "assistant",
            "content": partial_assistant_message.strip()
        }
    ],
    system=system_prompt,
    stop_sequences=["\n\nHuman:", "\n\nAssistant", "{function_calls}"]
).content[0].text

print(final_message)



The current dollar price is 82.25 INR
